# CODING TEST

**This was a coding test that I did using Python and SQL. To see my SQL code please go to the SQL folder.** <br /> 
The questions are as follows:<br /> 
    1) Count views, clicks, unique ads and unique campaigns by date.<br /> 
    2) Why there was a spike in events on 2019-04-05? Which events spiked? For all ads or particular?<br /> 
    3) Find top 10 ads by CTR. <br /> 
    Click Through Rate = Total count of clicks/Total count of views<br /> 
    4) Are the CTR of ads with video higher than the CTR of ads without video?<br /> 
    5) We need to report our revenue by days. What days were the most and the least profitable?
    We charge for ads if the ad cost type is CPC and the event is click. We also charge when the ad cost type is CPM and the event is view (cost is given per 1000 views).<br /> 
    6) What platform is the most popular among clients? <br /> 
    7) Are there ads that first were clicked and later viewed?<br />   

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:/Users/Yoga/Desktop/test/dataset.csv")

In [3]:
df.head()

,Unnamed: 0,date,time,event,platform,ad_id,user_id,campaign_id,ad_cost_type,ad_cost,has_video,target_audience_count
0,0,4/1/2019,3/31/2019 21:00,view,android,45061,34734,45061,CPM,200.6,0,1955269
1,1,4/1/2019,3/31/2019 21:00,view,web,121288,121288,121288,CPM,187.4,0,232011
2,2,4/1/2019,3/31/2019 21:01,view,android,102737,102535,102564,CPC,60.7,0,4410
3,3,4/1/2019,3/31/2019 21:01,view,android,107564,106914,107564,CPM,217.3,0,62711
4,4,4/1/2019,3/31/2019 21:01,view,android,4922,37,4400,CPC,60.1,0,1183501


**1) Count views, clicks, unique ads and unique campaigns by date.**

In [4]:
view = df[df['event']=='view'].groupby('date', as_index=False).agg({'event':'count'})
view

,date,event
0,4/1/2019,21782
1,4/2/2019,46572
2,4/3/2019,59023
3,4/4/2019,275092
4,4/5/2019,427386
5,4/6/2019,60967


In [5]:
click = df[df['event']=='click'].groupby('date',as_index=False).agg({'event':'count'})
click

,date,event
0,4/1/2019,291
1,4/2/2019,545
2,4/3/2019,460
3,4/4/2019,643
4,4/5/2019,92321
5,4/6/2019,14918


In [6]:
unique_ads = df.groupby('date')['ad_id','campaign_id'].nunique()
unique_ads

,ad_id,campaign_id
date,,
4/1/2019,150,149
4/2/2019,344,336
4/3/2019,360,352
4/4/2019,407,396
4/5/2019,465,442
4/6/2019,220,212


In [7]:
merged = view.merge(click, on = 'date', how = 'outer')

In [8]:
table = merged.merge(unique_ads, on ='date', how = 'outer')\
.rename(columns= {'event_x':'view','event_y':'click'})
table

,date,view,click,ad_id,campaign_id
0,4/1/2019,21782,291,150,149
1,4/2/2019,46572,545,344,336
2,4/3/2019,59023,460,360,352
3,4/4/2019,275092,643,407,396
4,4/5/2019,427386,92321,465,442
5,4/6/2019,60967,14918,220,212


**2) Why there was a spike in events on 2019-04-05? Which events spiked? For all ads or particular?**

In [9]:
five = df[df['date']=='4/5/2019']
five.head()

,Unnamed: 0,date,time,event,platform,ad_id,user_id,campaign_id,ad_cost_type,ad_cost,has_video,target_audience_count
22073,22073,4/5/2019,4/4/2019 18:00,view,android,2709,20,2464,CPM,191.5,0,165373
22074,22074,4/5/2019,4/4/2019 18:00,view,web,22490,3707,22490,CPM,197.0,0,5920993
22075,22075,4/5/2019,4/4/2019 18:00,view,android,22490,3707,22490,CPM,197.0,0,5920993
22076,22076,4/5/2019,4/4/2019 18:00,view,web,41063,25648,41063,CPM,188.1,0,39647
22077,22077,4/5/2019,4/4/2019 18:00,view,android,44999,34489,44999,CPM,195.1,0,618337


In [10]:
#Let's create dataframes for views and clicks on the 5th of April
five_view = five[five['event']=='view']
five_click = five[five['event']=='click']

In [11]:
#Let's count views and clicks and sort them from highest to lowest
view_5 = five_view.groupby(['campaign_id','ad_id','date'], as_index =False).agg({'event':'count'}).sort_values(by = 'event', ascending =False)
view_5.head(10)

,campaign_id,ad_id,date,event
397,112260,112583,4/5/2019,302811
367,107728,107729,4/5/2019,29724
125,28142,28142,4/5/2019,20872
369,107837,107837,4/5/2019,8338
198,38889,38892,4/5/2019,7986
183,37720,37720,4/5/2019,4861
285,45008,45008,4/5/2019,3145
317,46629,46629,4/5/2019,2302
57,18424,18425,4/5/2019,2274
238,42518,42518,4/5/2019,1885


In [12]:
click_5 = five_click.groupby(['campaign_id','ad_id','date'], as_index =False)\
.agg({'event':'count'}).sort_values(by = 'event', ascending =False)
click_5.head()

,campaign_id,ad_id,date,event
157,112260,112583,4/5/2019,91017
67,38889,38892,4/5/2019,451
61,37720,37720,4/5/2019,83
87,42518,42518,4/5/2019,57
15,18424,18425,4/5/2019,39


In [13]:
#Let's see top views and clicks
top_views_april_5 = view_5.merge(click_5,on=['campaign_id','ad_id','date'],how = 'outer')\
.rename(columns= {'event_x':'view','event_y':'click'})\
.sort_values(by=['view','click'], ascending=False)

In [56]:
top_views_april_5.head(10)
#We can see that the ad with ID 112583 had enormous amount of views and clicks, 
#which probably resulted from altered ad settings and budget.

,campaign_id,ad_id,date,view,click
0,112260,112583,4/5/2019,302811,91017.0
1,107728,107729,4/5/2019,29724,21.0
2,28142,28142,4/5/2019,20872,31.0
3,107837,107837,4/5/2019,8338,3.0
4,38889,38892,4/5/2019,7986,451.0
5,37720,37720,4/5/2019,4861,83.0
6,45008,45008,4/5/2019,3145,NaN
7,46629,46629,4/5/2019,2302,3.0
8,18424,18425,4/5/2019,2274,39.0
9,42518,42518,4/5/2019,1885,57.0


**3) Find top 10 ads by CTR.<br/>
Click Through Rate=Total count of clicks/Total count of views**

In [15]:
#Let's add two separate columns for views and clicks.
df['view'] = df['event'].apply(lambda x: 1 if x =='view' else 0)
df['click'] = df['event'].apply(lambda x: 1 if x == 'click' else 0)

In [16]:
df.head()

,Unnamed: 0,date,time,event,platform,ad_id,user_id,campaign_id,ad_cost_type,ad_cost,has_video,target_audience_count,view,click
0,0,4/1/2019,3/31/2019 21:00,view,android,45061,34734,45061,CPM,200.6,0,1955269,1,0
1,1,4/1/2019,3/31/2019 21:00,view,web,121288,121288,121288,CPM,187.4,0,232011,1,0
2,2,4/1/2019,3/31/2019 21:01,view,android,102737,102535,102564,CPC,60.7,0,4410,1,0
3,3,4/1/2019,3/31/2019 21:01,view,android,107564,106914,107564,CPM,217.3,0,62711,1,0
4,4,4/1/2019,3/31/2019 21:01,view,android,4922,37,4400,CPC,60.1,0,1183501,1,0


In [17]:
#Let's create a dataframe for CTR with total number of views and click for each ad
ctr = df.groupby(['ad_id', 'has_video'], as_index=False).agg({'view':'sum','click':'sum'})

In [18]:
#Let's add a column with calculated CTR
ctr['CTR'] = ctr.click/ctr.view

In [19]:
ctr.head()

,ad_id,has_video,view,click,CTR
0,2,0,101,1,0.009901
1,3,0,184,3,0.016304
2,1902,0,41,1,0.024390
3,2064,0,35,0,0.000000
4,2132,0,58,1,0.017241


In [20]:
ctr.sort_values(by='CTR', ascending = False).head(20)
# We can see some problems with the logs. Some ads have been clicked but not seen which is impossible. 

,ad_id,has_video,view,click,CTR
926,120431,0,0,35,inf
203,26204,0,0,6,inf
439,41500,0,0,20,inf
880,117364,0,0,7,inf
119,19223,0,0,7,inf
847,115825,0,0,4,inf
564,45418,0,0,3,inf
934,120536,0,0,6,inf
939,120796,0,0,1,inf
873,117164,0,19,6,0.315789


In [21]:
#Let's replace them with NaN and drop them, as they provide false information. Then we can see top 10 ads by CTR
ctr=ctr.replace([np.inf], np.nan).dropna()
top_10_ctr=ctr.sort_values(by='CTR', ascending = False).head(10)
top_10_ctr

,ad_id,has_video,view,click,CTR
873,117164,0,19,6,0.315789
787,112583,0,351802,105767,0.300644
463,42507,0,11,3,0.272727
665,98569,0,16,3,0.187500
621,46639,0,253,44,0.173913
174,23599,0,24,4,0.166667
126,19912,0,25,4,0.160000
756,110414,0,32,5,0.156250
136,20662,0,26,4,0.153846
593,45969,0,13,2,0.153846


**4)Are the CTR of ads with video higher than the CTR of ads without video?**

In [30]:
# Let's count how many ads in out dataset have a video and how many of them don't
has_video =ctr[ctr['has_video']==1]
has_video.nunique()

ad_id        20
has_video     1
view         18
click         7
CTR          13
dtype: int64

In [31]:
ctr[ctr['has_video']==0].nunique()

ad_id        936
has_video      1
view         377
click         40
CTR          381
dtype: int64

In [32]:
#Let's calculate a mean and a median for each group
mean_has_video = ctr.groupby('has_video',as_index=False).agg({'CTR':'mean'})

In [33]:
median_has_video = ctr.groupby('has_video', as_index = False).agg({'CTR':'median'})

In [34]:
mean_has_video.merge(median_has_video, on='has_video')\
.rename(columns = {'CTR_x':'mean','CTR_y':'median'})
# Overall, the ads with videos have better CTR. 
# However, the group sizes (20 with video and 936 without video) do not allow us draw such conclusions.

,has_video,mean,median
0,0,0.015745,0.002733
1,1,0.020145,0.009761


**5) We need to report our revenue by days. What days were the most and the least profitable? <br/>
We charge for ads if the ad cost type is CPC and the event is click.<br/> 
We also charge when the ad cost type is CPM and the event is view (cost is given per 1000 views).**


In [35]:
#Let's add two separate columns for CPC and CPM
df['CPC'] = df['ad_cost_type'].apply(lambda x: 1 if x =='CPC' else 0)
df['CPM'] = df['ad_cost_type'].apply(lambda x: 1 if x == 'CPM' else 0)

In [36]:
df.head()

,Unnamed: 0,date,time,event,platform,ad_id,user_id,campaign_id,ad_cost_type,ad_cost,has_video,target_audience_count,view,click,CPC,CPM
0,0,4/1/2019,3/31/2019 21:00,view,android,45061,34734,45061,CPM,200.6,0,1955269,1,0,0,1
1,1,4/1/2019,3/31/2019 21:00,view,web,121288,121288,121288,CPM,187.4,0,232011,1,0,0,1
2,2,4/1/2019,3/31/2019 21:01,view,android,102737,102535,102564,CPC,60.7,0,4410,1,0,1,0
3,3,4/1/2019,3/31/2019 21:01,view,android,107564,106914,107564,CPM,217.3,0,62711,1,0,0,1
4,4,4/1/2019,3/31/2019 21:01,view,android,4922,37,4400,CPC,60.1,0,1183501,1,0,1,0


In [37]:
#Knowing the formula for counting cost, we can count how much we have earned for each ad in a new column.
df['cost'] = (df.CPM*df.ad_cost*df.view)/1000 +(df.CPC*df.ad_cost*df.click)

In [38]:
df.head()

,Unnamed: 0,date,time,event,platform,ad_id,user_id,campaign_id,ad_cost_type,ad_cost,has_video,target_audience_count,view,click,CPC,CPM,cost
0,0,4/1/2019,3/31/2019 21:00,view,android,45061,34734,45061,CPM,200.6,0,1955269,1,0,0,1,0.2006
1,1,4/1/2019,3/31/2019 21:00,view,web,121288,121288,121288,CPM,187.4,0,232011,1,0,0,1,0.1874
2,2,4/1/2019,3/31/2019 21:01,view,android,102737,102535,102564,CPC,60.7,0,4410,1,0,1,0,0.0000
3,3,4/1/2019,3/31/2019 21:01,view,android,107564,106914,107564,CPM,217.3,0,62711,1,0,0,1,0.2173
4,4,4/1/2019,3/31/2019 21:01,view,android,4922,37,4400,CPC,60.1,0,1183501,1,0,1,0,0.0000


In [58]:
df.groupby('date',as_index=False).agg({'cost':'sum'}).sort_values(by='cost', ascending = False)
# We can see that April 5 was the most profitable day and April 1  - the least profitable

,date,cost
4,4/5/2019,96123.121699
3,4/4/2019,54988.801100
2,4/3/2019,14145.956900
5,4/6/2019,13346.930800
1,4/2/2019,13285.790200
0,4/1/2019,6655.707200


**6) What platform is the most popular among clients?**

In [40]:
platform_popularity = df.groupby('platform').agg({'view':'sum'})
platform_popularity

,view
platform,
android,445722
ios,267117
web,177983


In [41]:
#To count percentage we need the total number of views
total_views = df['view'].sum()

In [42]:
platform_popularity['percentage'] = platform_popularity.view/total_views*100

In [43]:
round(platform_popularity,1)
#Android accounts for 50% of ad views. It's the most popular platform

,view,percentage
platform,,
android,445722,50.0
ios,267117,30.0
web,177983,20.0


**7) Are there ads that first were clicked and later viewed?**

In [44]:
#Let's sort values in our dataframe by time
df = df.sort_values(by='time')

In [45]:
#Let's group them by ad_id, taking only the first events and first corresponding time
df2 = df.groupby(['campaign_id','ad_id'],as_index=False).agg({'time':'first','event':'first'})
df2.head()

,campaign_id,ad_id,time,event
0,1,2,3/31/2019 21:59,view
1,1,3,3/31/2019 21:08,view
2,1887,1902,4/2/2019 21:48,view
3,2064,2064,4/1/2019 21:52,view
4,2073,2132,4/2/2019 23:01,view


In [46]:
#Let's see how many ads have a 'click' as their first event
df2[df2['event']=='click'].nunique()

campaign_id    22
ad_id          22
time           22
event           1
dtype: int64

In [47]:
#Let's see all of them
clicked_first = df2[df2['event']=='click']

In [48]:
clicked_first

,campaign_id,ad_id,time,event
115,18539,18681,4/4/2019 21:18,click
119,19223,19223,4/1/2019 10:08,click
174,23595,23599,4/4/2019 21:05,click
203,26204,26204,3/31/2019 23:33,click
243,29864,29881,4/4/2019 21:03,click
285,33023,33033,4/4/2019 21:10,click
329,36758,36758,4/3/2019 22:21,click
359,38224,38224,4/3/2019 21:02,click
439,41498,41500,4/1/2019 10:59,click
522,44283,44283,4/3/2019 21:09,click
